### Custom Word Embeddings using Keras  

In [1]:
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding

Using TensorFlow backend.


Next, we need to define our dataset.

We will be using a very simple custom dataset that will contain reviews above movies.

The following script creates our dataset:

In [2]:
corpus = [
    # Positive Reviews

    'This is an excellent movie',
    'The move was fantastic I like it',
    'You should watch it is brilliant',
    'Exceptionally good',
    'Wonderfully directed and executed I like it',
    'Its a fantastic series',
    'Never watched such a brillent movie',
    'It is a Wonderful movie',

    # Negtive Reviews

    "horrible acting",
    'waste of money',
    'pathetic picture',
    'It was very boring',
    'I did not like the movie',
    'The movie was horrible',
    'I will not recommend',
    'The acting is pathetic'
]

Our corpus has 8 positive reviews and 8 negative reviews. 

The next step is to create label set for our data.

In [3]:
sentiments = array([1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0])

The first parameter to the Embedding() layer is the vocabulary, or number of unique words in the corpus. 

Let's first find the total number of words in our corpus:

In [4]:
from nltk.tokenize import word_tokenize

all_words = []
for sent in corpus:
    tokenize_word = word_tokenize(sent)
    for word in tokenize_word:
        all_words.append(word)

In [5]:
len(all_words)

71

We can retrieve all the unique words from a list by passing the list into the set function, as shown below.

In [6]:
unique_words = set(all_words)
print(len(unique_words))

45


In the output you will see "45", which is the number of unique words in our corpus.

The Embedding layer expects the words to be in numeric form. Therefore, we need to convert the sentences in our corpus to numbers.

One way to convert text to numbers is by using the one_hot function from the keras.preprocessing.text library. 

The function takes sentence and the total length of the vocabulary and returns the sentence in numeric form.

In [7]:
vocab_length = 45
embedded_sentences = [one_hot(sent, vocab_length) for sent in corpus]
print(embedded_sentences )

[[2, 26, 4, 13, 31], [7, 26, 12, 36, 40, 11, 36], [2, 43, 3, 36, 26, 8], [9, 37], [35, 29, 34, 16, 40, 11, 36], [23, 25, 36, 10], [4, 36, 12, 25, 10, 31], [36, 26, 25, 8, 31], [22, 9], [36, 21, 14], [42, 9], [36, 12, 8, 39], [40, 11, 13, 11, 7, 31], [7, 31, 12, 22], [40, 3, 13, 25], [7, 9, 26, 42]]


The embedding layer expects sentences to be of equal size. 

However, our encoded sentences are of different sizes.

One way to make all the sentences of uniform size is to increase the lenght of all the sentences and make it equal to the length of the largest sentence. 

Let's first find the largest sentence in our corpus and then we will increase the length of all the sentences to the length of the largest sentence. To do so, execute the following script:

In [8]:
word_count = lambda sentence: len(word_tokenize(sentence))
longest_sentence = max(corpus, key=word_count)
length_long_sentence = len(word_tokenize(longest_sentence))

In [9]:
length_long_sentence

7

Next to make all the sentences of equal size, we will add zeros to the empty indexes that will be created as a result of increasing the sentence length

To append the zeros at the end of the sentencses, we can use the pad_sequences method.

The first parameter is the list of encoded sentences of unequal sizes, the second parameter is the size of the longest sentence or the padding index, while the last parameter is padding where you specify post to add padding at the end of sentences.

In [11]:
padded_sentences = pad_sequences(embedded_sentences, length_long_sentence, padding='post')
print(padded_sentences)

[[ 2 26  4 13 31  0  0]
 [ 7 26 12 36 40 11 36]
 [ 2 43  3 36 26  8  0]
 [ 9 37  0  0  0  0  0]
 [35 29 34 16 40 11 36]
 [23 25 36 10  0  0  0]
 [ 4 36 12 25 10 31  0]
 [36 26 25  8 31  0  0]
 [22  9  0  0  0  0  0]
 [36 21 14  0  0  0  0]
 [42  9  0  0  0  0  0]
 [36 12  8 39  0  0  0]
 [40 11 13 11  7 31  0]
 [ 7 31 12 22  0  0  0]
 [40  3 13 25  0  0  0]
 [ 7  9 26 42  0  0  0]]


Now we have everything that we need to create a sentiment classification model using word embeddings.

We will create a very simple text classification model with an embedding layer and no hidden layers. 

Look at the following script:

In [17]:
model = Sequential()
model.add(Embedding(vocab_length, 20, input_length=length_long_sentence))
model.add(Flatten())

model.add(Dense(1, activation='sigmoid'))

In the script above, we create a Sequential model and add the Embedding layer as the first layer to the model.

The length of the vocabulary is specified by the vocab_length parameter. 

The dimension of each word vector will be 20 and the input_length will be the length of the longest sentence, which is 7. 

Next, the Embedding layer is flattened so that it can be directly used with the densely connected layer. 

Since it is a binary classification problem, we use the sigmoid function as the loss function at the dense layer.

Next, we will compile the model and print the summary of our model, as shown below:

In [18]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 7, 20)             900       
_________________________________________________________________
flatten_2 (Flatten)          (None, 140)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 141       
Total params: 1,041
Trainable params: 1,041
Non-trainable params: 0
_________________________________________________________________
None


You can see that the first layer has 900 trainable parameters. 

This is because our vocabulary size is 45 and each word will be presented as a 20 dimensional vector.

Hence the total number of trainable parameters will be 900. 

Similarly, the output from the embedding layer will be a sentence with 7 words where each word is represented by a 20 dimensional vector. 

However, when the 2D output is flattened, we get a 140 dimensional vector (7 x 20). The flattened vector is directly connected to the dense layer that contains 1 neuron.

Now let's train the model on our data using the fit method, as shown below:

In [19]:
model.fit(padded_sentences, sentiments, epochs=100)

/home/renuk/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/100
16/16 [==============================] - 0s 6ms/step - loss: 0.6971 - acc: 0.4375
Epoch 2/100
16/16 [==============================] - 0s 74us/step - loss: 0.6933 - acc: 0.5000
Epoch 3/100
16/16 [==============================] - 0s 145us/step - loss: 0.6902 - acc: 0.5000
Epoch 4/100
16/16 [==============================] - 0s 88us/step - loss: 0.6868 - acc: 0.5625
Epoch 5/100
16/16 [==============================] - 0s 92us/step - loss: 0.6834 - acc: 0.6875
Epoch 6/100
16/16 [==============================] - 0s 95us/step - loss: 0.6799 - acc: 0.6875
Epoch 7/100
16/16 [==============================] - 0s 118us/step - loss: 0.6763 - acc: 0.8125
Epoch 8/100
16/16 [==============================] - 0s 138us/step - loss: 0.6728 - acc: 0.8125
Epoch 9/100
16/16 [==============================] - 0s 131us/step - loss: 0.6692 - acc: 0.8125
Epoch 10/100
16/16 [==============================] - 0s 306us/step - loss: 0.6656 - acc: 0.8125
Epoch 11/100
16/16 [=========================

Epoch 86/100
16/16 [==============================] - 0s 136us/step - loss: 0.3455 - acc: 0.9375
Epoch 87/100
16/16 [==============================] - 0s 148us/step - loss: 0.3413 - acc: 0.9375
Epoch 88/100
16/16 [==============================] - 0s 109us/step - loss: 0.3372 - acc: 0.9375
Epoch 89/100
16/16 [==============================] - 0s 500us/step - loss: 0.3331 - acc: 0.9375
Epoch 90/100
16/16 [==============================] - 0s 243us/step - loss: 0.3290 - acc: 0.9375
Epoch 91/100
16/16 [==============================] - 0s 125us/step - loss: 0.3250 - acc: 0.9375
Epoch 92/100
16/16 [==============================] - 0s 130us/step - loss: 0.3210 - acc: 0.9375
Epoch 93/100
16/16 [==============================] - 0s 131us/step - loss: 0.3170 - acc: 0.9375
Epoch 94/100
16/16 [==============================] - 0s 496us/step - loss: 0.3131 - acc: 0.9375
Epoch 95/100
16/16 [==============================] - 0s 117us/step - loss: 0.3092 - acc: 0.9375
Epoch 96/100
16/16 [==========

In [15]:
# We will train and test the model using the same corpus. 

# Execute the following script to evaluate the model performance on our corpus:
    
    

In [16]:
loss, accuracy = model.evaluate(padded_sentences, sentiments, verbose=0)
print('Accuracy: %f' % (accuracy*100))

Accuracy: 100.000000


###  Loading Pretrained Word Embeddings

In the previous section we trained custom word embeddings. However, we can also use pretrained word embeddings.

Several types of pretrained word embeddings exist, however we will be using the GloVe word embeddings from Stanford NLP since it is the most famous one and commonly used. The word embeddings can be downloaded from this link.

The smallest file is named "Glove.6B.zip". The size of the file is 822 MB. The file contains 50, 100, 200, and 300 dimensional word vectors for 400k words. We will be using the 100 dimensional vector.

The process is quite similar. First we have to import the required libraries:

In [21]:
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding

In [18]:
corpus = [
    # Positive Reviews

    'This is an excellent movie',
    'The move was fantastic I like it',
    'You should watch it is brilliant',
    'Exceptionally good',
    'Wonderfully directed and executed I like it',
    'Its a fantastic series',
    'Never watched such a brillent movie',
    'It is a Wonderful movie',

    # Negtive Reviews

    "horrible acting",
    'waste of money',
    'pathetic picture',
    'It was very boring',
    'I did not like the movie',
    'The movie was horrible',
    'I will not recommend',
    'The acting is pathetic'
]

In [19]:
sentiments = array([1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0])

In the last section, we used one_hot function to convert text to vectors. 

Another approach is to use Tokenizer function from keras.preprocessing.text library.

You simply have to pass your corpus to the Tokenizer's fit_on_text method.

In [22]:
word_tokenizer = Tokenizer()
word_tokenizer.fit_on_texts(corpus)

To get the number of unique words in the text, you can simply count the length of word_index dictionary of the word_tokenizer object. 

Remember to add 1 with the vocabulary size. 

This is to store the dimensions for the words for which no pretrained word embeddings exist.

In [25]:
vocab_length = len(word_tokenizer.word_index) + 1
vocab_length

44

Finally, to convert sentences to their numeric counterpart, call the texts_to_sequences function and pass it the whole corpus.

In [24]:
embedded_sentences = word_tokenizer.texts_to_sequences(corpus)
print(embedded_sentences)

[[14, 3, 15, 16, 1], [4, 17, 6, 9, 5, 7, 2], [18, 19, 20, 2, 3, 21], [22, 23], [24, 25, 26, 27, 5, 7, 2], [28, 8, 9, 29], [30, 31, 32, 8, 33, 1], [2, 3, 8, 34, 1], [10, 11], [35, 36, 37], [12, 38], [2, 6, 39, 40], [5, 41, 13, 7, 4, 1], [4, 1, 6, 10], [5, 42, 13, 43], [4, 11, 3, 12]]


The next step is to find the number of words in the longest sentence 
and then to apply padding to the sentences having shorter lengths than the length of the longest sentence.



In [28]:
from nltk.tokenize import word_tokenize

word_count = lambda sentence: len(word_tokenize(sentence))
longest_sentence = max(corpus, key=word_count)
length_long_sentence = len(word_tokenize(longest_sentence))

padded_sentences = pad_sequences(embedded_sentences, length_long_sentence, padding='post')

print(padded_sentences)

[[14  3 15 16  1  0  0]
 [ 4 17  6  9  5  7  2]
 [18 19 20  2  3 21  0]
 [22 23  0  0  0  0  0]
 [24 25 26 27  5  7  2]
 [28  8  9 29  0  0  0]
 [30 31 32  8 33  1  0]
 [ 2  3  8 34  1  0  0]
 [10 11  0  0  0  0  0]
 [35 36 37  0  0  0  0]
 [12 38  0  0  0  0  0]
 [ 2  6 39 40  0  0  0]
 [ 5 41 13  7  4  1  0]
 [ 4  1  6 10  0  0  0]
 [ 5 42 13 43  0  0  0]
 [ 4 11  3 12  0  0  0]]


We have converted our sentences into padded sequence of numbers. 

The next step is to load the GloVe word embeddings and 

then create our embedding matrix that contains the words in our corpus and their corresponding values from GloVe embeddings. Run the following script:

In [37]:
from numpy import array
from numpy import asarray
from numpy import zeros

embeddings_dictionary = dict()
glove_file = open('/home/renuk/Documents/Deep_learning/notebooks/glove.6B/glove.6B.100d.txt', encoding="utf8")

In the script above, in addition to loading the GloVe embeddings, we also imported a few libraries.

We will see the use of these libraries in the upcoming section. 

Here notice that we loaded glove.6B.100d.txt file. This file contains 100 dimensional word embeddings. 

We also created an empty dictionary that will store our word embeddings.

If you open the file, you will see a word at the beginning of each line followed by set of 100 numbers. 

The numbers form the 100 dimensional vector for the word at the begining of each line.

We will create a dictionary that will contain words as keys and the corresponding 100 dimensional vectors as values, in the form of an array. Execute the following script:

In [38]:
for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = asarray(records[1:], dtype='float32')
    embeddings_dictionary [word] = vector_dimensions

glove_file.close()

The dictionary embeddings_dictionary now contains words and corresponding GloVe embeddings for all the words.

We want the word embeddings for only those words that are present in our corpus. We will create a two dimensional numpy array of 44 (size of vocabulary) rows and 100 columns. The array will initially contain zeros. The array will be named as embedding_matrix

Next, we will iterate through each word in our corpus by traversing the word_tokenizer.word_index dictionary that contains our words and their corresponding index.

Each word will be passed as key to the embedding_dictionary to retrieve the corresponding 100 dimensional vector for the word. The 100 dimensional vector will then be stored at the corresponding index of the word in the embedding_matrix. Look at the following script:

In [39]:
embedding_matrix = zeros((vocab_length, 100))
for word, index in word_tokenizer.word_index.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector


Our embedding_matrix now contains pretrained word embeddings for the words in our corpus.

Now we are ready to create our sequential model. Look at the following script:



In [40]:
model = Sequential()
embedding_layer = Embedding(vocab_length, 100, weights=[embedding_matrix], input_length=length_long_sentence, trainable=False)
model.add(embedding_layer)
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

The script remains the same, except for the embedding layer. Here in the embedding layer, the first parameter is the size of the vacabulary. The second parameter is the vector dimension of the output vector. Since we are using pretrained word embeddings that contain 100 dimensional vector, we set the vector dimension to 100.

Another very important attribute of the Embedding() layer that we did not use in the last section is weights. You can pass your pretrained embedding matrix as default weights to the weights parameter. And since we are not training the embedding layer, the trainable attribute has been set to False.

Let's compile our model and see the summary of our model:

In [41]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 7, 100)            4400      
_________________________________________________________________
flatten_2 (Flatten)          (None, 700)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 701       
Total params: 5,101
Trainable params: 701
Non-trainable params: 4,400
_________________________________________________________________
None


In [42]:
model.fit(padded_sentences, sentiments, epochs=100, verbose=1)

Epoch 1/100
16/16 [==============================] - 0s 6ms/step - loss: 0.8417 - acc: 0.3750
Epoch 2/100
16/16 [==============================] - 0s 103us/step - loss: 0.8181 - acc: 0.3750
Epoch 3/100
16/16 [==============================] - 0s 80us/step - loss: 0.7923 - acc: 0.3750
Epoch 4/100
16/16 [==============================] - 0s 83us/step - loss: 0.7667 - acc: 0.5000
Epoch 5/100
16/16 [==============================] - 0s 74us/step - loss: 0.7419 - acc: 0.5000
Epoch 6/100
16/16 [==============================] - 0s 78us/step - loss: 0.7182 - acc: 0.5625
Epoch 7/100
16/16 [==============================] - 0s 151us/step - loss: 0.6954 - acc: 0.5625
Epoch 8/100
16/16 [==============================] - 0s 121us/step - loss: 0.6737 - acc: 0.5625
Epoch 9/100
16/16 [==============================] - 0s 92us/step - loss: 0.6527 - acc: 0.5625
Epoch 10/100
16/16 [==============================] - 0s 197us/step - loss: 0.6325 - acc: 0.5625
Epoch 11/100
16/16 [==========================

Epoch 86/100
16/16 [==============================] - 0s 141us/step - loss: 0.1076 - acc: 1.0000
Epoch 87/100
16/16 [==============================] - 0s 172us/step - loss: 0.1060 - acc: 1.0000
Epoch 88/100
16/16 [==============================] - 0s 132us/step - loss: 0.1044 - acc: 1.0000
Epoch 89/100
16/16 [==============================] - 0s 86us/step - loss: 0.1029 - acc: 1.0000
Epoch 90/100
16/16 [==============================] - 0s 138us/step - loss: 0.1014 - acc: 1.0000
Epoch 91/100
16/16 [==============================] - 0s 130us/step - loss: 0.0999 - acc: 1.0000
Epoch 92/100
16/16 [==============================] - 0s 162us/step - loss: 0.0985 - acc: 1.0000
Epoch 93/100
16/16 [==============================] - 0s 95us/step - loss: 0.0971 - acc: 1.0000
Epoch 94/100
16/16 [==============================] - 0s 106us/step - loss: 0.0957 - acc: 1.0000
Epoch 95/100
16/16 [==============================] - 0s 218us/step - loss: 0.0944 - acc: 1.0000
Epoch 96/100
16/16 [============